# Program Assignment #3 - K-means clustering & Support Vector Machine
---

## Name: 李勝維
## Student ID: 0711239
---

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(0)